In [1]:
import sys, os
import numpy as np
import faiss
import torch
from ai import *
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer, AutoModel, pipeline
from tqdm import tqdm

In [2]:
# Автоперезагрузка файлов
%load_ext autoreload
%autoreload 2

In [3]:
os.environ["PYTHONDONTWRITEBYTECODE"] = "1"

Load AI

In [4]:
ai = AI()
ai.database_path = os.path.abspath("../../var/database")
ai.device = "cpu"
ai.load_store()
ai.init_llm()

Send question

In [ ]:
history = []
question = "Что такое облачная BAYRELL Cloud OS?"
#question = "Как ее установить BAYRELL Cloud OS?"

# Получить промт
result = await ai.get_prompt(history, question)
#ai.print_documents(result["docs"])
print(result["prompt"])

# Отправим вопрос LLM
for chunk in ai.llm_answer.stream(result["prompt"], stream=True):
    print(chunk.content, end="", flush=True)

In [ ]:
history = [
    {
        "sender": "human",
        "text": "Что такое облачная BAYRELL Cloud OS?",
    }
]
question = "Как его установить на сервер?"
result = await ai.get_prompt(history, question)
#ai.print_documents(result["docs"])

# Отправим вопрос LLM
for chunk in ai.llm_answer.stream(result["prompt"], stream=True):
    print(chunk.content, end="", flush=True)

In [ ]:
# Загружаем модель
qa_pipeline = pipeline("question-answering", model="DeepPavlov/rubert-base-cased-squad2")

In [ ]:
def filter_docs_qa(docs, question):
    relevant_docs = []
    for doc in docs:
        result = qa_pipeline(question=question, context=doc.page_content)
        if result['score'] > 0.5:  # Фильтр по уверенности
            relevant_docs.append(doc)
    return relevant_docs

In [ ]:
# Запрос и кандидаты
query = "Что такое Облачная ОС?"
documents = [
    "Облачная ОС — это операционная система, работающая в облаке.",
    "Компьютерная сеть — это соединение устройств для обмена данными.",
    "Linux — это операционная система с открытым кодом."
]

# Формируем пары (query, document)
pairs = [(query, doc) for doc in documents]

reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
#reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
#reranker = CrossEncoder('cross-encoder/stsb-TinyBERT-L-2')

# Предсказываем релевантность (чем выше — тем лучше)
scores = reranker.predict(pairs)

# Сортируем документы по убыванию оценки
ranked_docs = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

# Выводим результаты
for i, (doc, score) in enumerate(ranked_docs):
    print(f"Rank {i+1}: {doc} (Score: {score:.4f})")


In [ ]:
class Test1:
    def __init__(self, model_name):
        self.model = HuggingFaceEmbeddings(
            model_name=model_name
        )
        self.model_name = model_name
        
    def get_embeddings(self, text):
        embeddings = self.model.embed_documents([text])
        return np.array(embeddings[0])


class Test2:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model_name = model_name

    def get_embeddings(self, text):
        t = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**{k: v.to(self.model.device) for k, v in t.items()})
        #embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = model_output.last_hidden_state.mean(dim=1)
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu().numpy()


model_name = "cointegrated/rubert-tiny2"
test1 = Test1(model_name)
test2 = Test2(model_name)

text = "Hello world"
emb1 = test1.get_embeddings(text)
emb2 = test2.get_embeddings(text)
print(emb1 == emb2)